In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


"""
# Create a SparkSession
spark = SparkSession.builder \
    .appName("Load MySQL Table and Write as Delta") \
    .getOrCreate()
#     .config("spark.jars", "/path/to/mysql-connector-java.jar") \
spark.sparkContext.setJobDescription("delta job")
"""

from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession

builder = (
        SparkSession.builder
        .appName("pyspark-deltaLake-local")
        .config('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')
        .config('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog')

)

spark = configure_spark_with_delta_pip(builder).getOrCreate()


In [11]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [12]:
import findspark
findspark.init()
findspark.add_packages('mysql:mysql-connector-java:9.2.0')
print( spark.version )

3.5.4


In [13]:
sample_data = [{"name": "John    D.", "age": 30},
  {"name": "Alice   G.", "age": 25},
  {"name": "Bob  T.", "age": 35},
  {"name": "Eve   A.", "age": 28}]

df = spark.createDataFrame(sample_data)
df.show()

+---+----------+
|age|      name|
+---+----------+
| 30|John    D.|
| 25|Alice   G.|
| 35|   Bob  T.|
| 28|  Eve   A.|
+---+----------+



In [14]:
import configparser

# Create a ConfigParser object
config = configparser.ConfigParser()

# Read the .ini file
config.read('settings.ini')  # Replace with the path to your .ini file
# Accessing data from the .ini file
username = config['settings']['username']
password = config['settings']['password']
host = config['settings']['host']
port = config['settings']['port']
schema = config['database']['schema']
print('loaded ini')


loaded ini


In [15]:

# Define the MySQL connection properties
url = f"jdbc:mysql://{host}:{port}/{schema}"  # Replace with your MySQL server details
# print( url ) 
properties = {
    "user": username,  # Replace with your MySQL username
    "password": password,  # Replace with your MySQL password
    "driver": "com.mysql.cj.jdbc.Driver"  # MySQL JDBC driver
}


# Load data from MySQL table into a PySpark DataFrame
df = spark.read.jdbc(url=url, table="flights", properties=properties)

# Show the data
df.show(5)

+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
| id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
|  0|2013|    1|  1|   517.0|           515|      2.0|   830.0|           819|     11.0|     UA|  1545| N14228|   EWR| IAH|   227.0|    1400|   5|    15|2013-01-01 05:00:00|United Air Lines ...|
|  1|2013|    1|  1|   533.0|           529|      4.0|   850.0|           830|     20.0|     UA|  1714| N24211|   LGA| IAH|   227.0|    1416|   5|    29|2013-01-01 05:00:00|United Air Lines ...|
|  2|2013|    1|  1|   54

In [29]:
object_name = "flights"
# Load data from the MySQL table into a PySpark DataFrame
df = spark.read.jdbc(url=url, table=object_name, properties=properties)

# Register the DataFrame as a temporary view
df.createOrReplaceTempView( object_name )

# Show the result of the SQL query
df.show(5)
query_sql = "SELECT * FROM flights WHERE month = 1"
result_df = spark.sql(query_sql)

# Define the path where the Delta table will be saved
delta_path = "data/flights_merge/" # Replace with your desired Delta table path

# Save the DataFrame as a Delta table
result_df.write.format("delta").mode("overwrite").save(delta_path)



+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
| id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
|  0|2013|    1|  1|   517.0|           515|      2.0|   830.0|           819|     11.0|     UA|  1545| N14228|   EWR| IAH|   227.0|    1400|   5|    15|2013-01-01 05:00:00|United Air Lines ...|
|  1|2013|    1|  1|   533.0|           529|      4.0|   850.0|           830|     20.0|     UA|  1714| N24211|   LGA| IAH|   227.0|    1416|   5|    29|2013-01-01 05:00:00|United Air Lines ...|
|  2|2013|    1|  1|   54

In [25]:

query_sql = "SELECT * FROM flights WHERE month <= 2"
two_month_df = spark.sql(query_sql)
two_month_df.show(5)

+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
| id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
|  0|2013|    1|  1|   517.0|           515|      2.0|   830.0|           819|     11.0|     UA|  1545| N14228|   EWR| IAH|   227.0|    1400|   5|    15|2013-01-01 05:00:00|United Air Lines ...|
|  1|2013|    1|  1|   533.0|           529|      4.0|   850.0|           830|     20.0|     UA|  1714| N24211|   LGA| IAH|   227.0|    1416|   5|    29|2013-01-01 05:00:00|United Air Lines ...|
|  2|2013|    1|  1|   54

In [30]:
from delta.tables import DeltaTable

delta_name = "flights"
# Define the path to the existing Delta table
delta_path = "data/flights_merge/"  

# Load the existing Delta table
delta_table = DeltaTable.forPath(spark, delta_path)
delta_table_df = delta_table.toDF()
delta_table_df.show(5)
df.createOrReplaceTempView( "flights_merge" )

+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
| id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
|  0|2013|    1|  1|   517.0|           515|      2.0|   830.0|           819|     11.0|     UA|  1545| N14228|   EWR| IAH|   227.0|    1400|   5|    15|2013-01-01 05:00:00|United Air Lines ...|
|  1|2013|    1|  1|   533.0|           529|      4.0|   850.0|           830|     20.0|     UA|  1714| N24211|   LGA| IAH|   227.0|    1416|   5|    29|2013-01-01 05:00:00|United Air Lines ...|
|  2|2013|    1|  1|   54

In [31]:
prevCheckpoint = spark.sql( "select max( month ) , max( day ) from flights_merge ")
prevCheckpoint.show() 

+----------+--------+
|max(month)|max(day)|
+----------+--------+
|        12|      31|
+----------+--------+



In [ ]:

# Define t`he condition for the merge with multiple columns
merge_condition = """source.month = target.month 
AND source.day = target.day
AND source.flight = target.flight
AND source.tailnum = target.tailnum"""

# Perform the merge operation
delta_table.alias("target").merge(
    two_month_df.alias("source"),
    merge_condition
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute() 

# Show the merged Delta table
delta_table.toDF().show(5)
df.createOrReplaceTempView( object_name )

+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
| id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
|159|2013|    1|  1|   856.0|           900|     -4.0|  1226.0|          1220|      6.0|     AA|     1| N324AA|   JFK| LAX|   358.0|    2475|   9|     0|2013-01-01 09:00:00|American Airlines...|
|295|2013|    1|  1|  1155.0|          1200|     -5.0|  1517.0|          1510|      7.0|     AA|     3| N322AA|   JFK| LAX|   353.0|    2475|  12|     0|2013-01-01 12:00:00|American Airlines...|
|823|2013|    1|  1|  222